<a href="https://colab.research.google.com/github/kiril-buga/Neural-Network-Training-Project/blob/main/1D_CNN_for_multi_label_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wfdb
# ===== Imports =====
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
USE_HF = True

In [ ]:
from huggingface_hub import snapshot_download

local_dir = snapshot_download(
    repo_id="Neural-Network-Project/ECG-database",
    repo_type="dataset"
)
print("Downloaded to:", local_dir)

Fetching 21 files:   0%|          | 0/21 [00:00<?, ?it/s]

Downloaded to: /root/.cache/huggingface/hub/datasets--Neural-Network-Project--ECG-database/snapshots/37c09929463534a0e5cea0f1b81b10e5d017466e


In [ ]:
if USE_HF and local_dir:
  # Case 2: You want to download the dataset from Huggingface
    DATA_PATH = f"{local_dir}/data/"
    ARTIFACT_DIR = f"{local_dir}/artifacts/"

else:
  # ===== Detect if running in Google Colab and mount Drive =====
  IN_COLAB = False
  try:
      from google.colab import drive  # type: ignore
      IN_COLAB = True
  except Exception:
      drive = None
      IN_COLAB = False

  if IN_COLAB:
      drive.mount('/content/drive/')

  # ===== Define paths =====
  if IN_COLAB:
      # Case 1: You manually placed the dataset in MyDrive
      DATA_PATH = "/content/drive/MyDrive/DeepLearningECG/data/"
      ARTIFACT_DIR = "/content/drive/MyDrive/DeepLearningECG/artifacts/"

  else:
      # Case 3: Local fallback (if running outside Colab)
      DATA_PATH = "../data/"
      ARTIFACT_DIR = "../artifacts/"

print("DATA_PATH:", DATA_PATH)
print("ARTIFACT_DIR:", ARTIFACT_DIR)
print("Files in DATA_PATH:", os.listdir(DATA_PATH))

DATA_PATH: /root/.cache/huggingface/hub/datasets--Neural-Network-Project--ECG-database/snapshots/37c09929463534a0e5cea0f1b81b10e5d017466e/data/
ARTIFACT_DIR: /root/.cache/huggingface/hub/datasets--Neural-Network-Project--ECG-database/snapshots/37c09929463534a0e5cea0f1b81b10e5d017466e/artifacts/
Files in DATA_PATH: ['ECGCode.csv', 'ExampleReadingCode.ipynb', 'Child_ecg.zip', 'AttributesDictionary.csv', 'Child_ecg_full.zip', 'Child_ecg.z01', 'Child_ecg', 'DiseaseCode.csv']


In [ ]:


# Install p7zip-full for handling various archive formats
!apt-get install -y p7zip-full

zip_base_name = "Child_ecg"
output_zip_path = os.path.join(DATA_PATH, f"{zip_base_name}.zip") # 7z can usually handle the first part directly

# Unzip the concatenated file using 7z
# -aoa option overwrites existing files without prompt
!7z x {output_zip_path} -o{DATA_PATH} -aoa

print("DONE")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /root/.cache/huggingface/hub/da . 0e5cea0f1b81b10e5d017466e/data/                                                                           1 file, 1908237306 bytes (1820 MiB)

Extracting archive: /root/.cache/huggingface/hub/datasets--Neural-Network-Project--ECG-database/snapshots/37c09929463534a0e5cea0f1b81b10e5d017466e/data/Child_ecg.zip
 10% 4096 Open              

In [ ]:
# Check if it was unzipped -> P00, P01...
!ls -lh {DATA_PATH+"Child_ecg"}

total 324K
drwxr-xr-x 1001 root root 24K Apr  9  2025 P00
drwx------ 1002 root root 24K Apr  9  2025 P01
drwx------ 1002 root root 24K Apr  9  2025 P02
drwx------ 1002 root root 32K Apr  9  2025 P03
drwx------ 1002 root root 24K Apr  9  2025 P04
drwx------ 1002 root root 28K Apr  9  2025 P05
drwx------ 1002 root root 24K Apr  9  2025 P06
drwx------ 1002 root root 24K Apr  9  2025 P07
drwx------ 1002 root root 24K Apr  9  2025 P08
drwx------ 1002 root root 28K Apr  9  2025 P09
drwx------ 1002 root root 28K Apr  9  2025 P10
drwx------  646 root root 20K Apr  9  2025 P11


In [ ]:
import os
import pandas as pd
import wfdb
import numpy as np
from sklearn.model_selection import train_test_split


Y = pd.read_csv(os.path.join(DATA_PATH, "AttributesDictionary.csv"))
print(Y.head())

                Filename      ECG_ID Patient_ID    Age    Gender  \
0  P00/P00001/P00001_E01  P00001_E01     P00001   572d  'Female'   
1  P00/P00002/P00002_E01  P00002_E01     P00002  4327d    'Male'   
2  P00/P00003/P00003_E01  P00003_E01     P00003  1087d  'Female'   
3  P00/P00004/P00004_E01  P00004_E01     P00004  2465d    'Male'   
4  P00/P00004/P00004_E02  P00004_E02     P00004  2461d    'Male'   

      Acquisition_date  Sampling_point  Lead  \
0  2017-11-22 10:46:08            9000     9   
1  2017-11-28 21:59:47           15000    12   
2  2017-11-29 16:04:57           10000    12   
3  2017-11-30 15:21:27           13000     9   
4  2017-11-26 19:19:48           15000     9   

                                 AHA_code       CHN_code  \
0  'Left ventricular high voltage';'L147'  'J106';'L123'   
1                                   'C21'          'C13'   
2                                   'C21'          'C13'   
3                                   'C21'          'C13'   
4 

In [ ]:
def load_raw_data(df, path):
    filenames = df['Filename']
    data = [wfdb.rdsamp(os.path.join(path, f))[0] for f in filenames]
    return data

def load_Diag(df, path):
    Disease_Diag = []
    ECG_Diag = []
    for filename in df['Filename']:
        record = wfdb.rdrecord(os.path.join(path, filename))
        message = record.comments
        Disease_Diag.append(message[1])
        ECG_Diag.append(message[2])
    return Disease_Diag, ECG_Diag

records = DATA_PATH +"Child_ecg"
X = load_raw_data(Y, records)                 # list of np arrays, one per record
Disease_Diag, ECG_Diag = load_Diag(Y, records)
